In [1]:
%%cpp -d
//RNTUPLE headers
#include <ROOT/RNTuple.hxx>
#include <ROOT/RNTupleModel.hxx>
#include "nlohmann/json.hpp"
#include <cstdio>


In [2]:
//copy past from SZ3/Config.hpp
/*
namespace SZ3{
    enum EB {
        EB_ABS, EB_REL, EB_PSNR, EB_L2NORM, EB_ABS_AND_REL, EB_ABS_OR_REL
    };
    constexpr const char *EB_STR[] = {"ABS", "REL", "PSNR", "NORM", "ABS_AND_REL", "ABS_OR_REL"};
    constexpr EB EB_OPTIONS[] = {EB_ABS, EB_REL, EB_PSNR, EB_L2NORM, EB_ABS_AND_REL, EB_ABS_OR_REL};

    enum ALGO {
        ALGO_LORENZO_REG, ALGO_INTERP_LORENZO, ALGO_INTERP
    };
    constexpr const char *ALGO_STR[] = {"ALGO_LORENZO_REG", "ALGO_INTERP_LORENZO", "ALGO_INTERP"};
    constexpr const ALGO ALGO_OPTIONS[] = {ALGO_LORENZO_REG, ALGO_INTERP_LORENZO, ALGO_INTERP};

    enum INTERP_ALGO {
        INTERP_ALGO_LINEAR, INTERP_ALGO_CUBIC
    };
    constexpr const char *INTERP_ALGO_STR[] = {"INTERP_ALGO_LINEAR", "INTERP_ALGO_CUBIC"};
    constexpr INTERP_ALGO INTERP_ALGO_OPTIONS[] = {INTERP_ALGO_LINEAR, INTERP_ALGO_CUBIC};

}
*/
std::string fname = "/home/abashyal/git_compression/compressiontests/test_SZ3_compression.root";

In [3]:
using namespace ROOT::Experimental;

In [4]:
auto model = RNTupleModel::Create();

In [5]:
auto reader = RNTupleReader::Open(std::move(model),"Compressions",fname.c_str());

Warning in <[ROOT.NTuple] Warning /home/conda/feedstock_root/build_artifacts/root_base_1707256713696/work/root-source/tree/ntuple/v7/src/RNTupleSerialize.cxx:1398 in static ROOT::Experimental::RResult<void> ROOT::Experimental::Internal::RNTupleSerializer::DeserializeHeaderV1(const void*, uint32_t, ROOT::Experimental::RNTupleDescriptorBuilder&)>: Pre-release format version: RC 1


In [6]:
reader->PrintInfo();

************************************ NTUPLE ************************************
* N-Tuple : Compressions                                                       *
* Entries : 18000                                                              *
********************************************************************************
* Field 1       : out_data (std::vector<double>)                               *
*   Field 1.1   : _0 (double)                                                  *
* Field 2       : compressed_data (std::vector<char>)                          *
*   Field 2.1   : _0 (char)                                                    *
* Field 3       : parameters (std::string)                                     *
********************************************************************************


In [7]:
auto params = reader->GetView<std::string>("parameters");
auto nentries = reader->GetNEntries();

In [8]:
nlohmann::json jobj = nlohmann::json::parse(params(0));

In [9]:
std::cout<<jobj.dump()<<std::endl;

{"ABS_ERROR_BOUND":0.005,"ALGO_OPTIONS":0,"Algorithm":"SZ3","COMPRESSION_RATIO":1328.0,"EB_OPTIONS":0,"INTERP_ALGO":0,"L2NORM_ERROR_BOUND":0.005,"PSNR_ERROR_BOUND":0.005,"Par_name":"ALGO_OPTIONS, EB_OPTIONS, INTERP_ALGO,ERROR_BOUND_TYPE,ABS_ERROR_BOUND,REL_ERROR_BOUND,PSNR_ERROR_BOUND,L2NORM_ERROR_BOUND","REL_ERROR_BOUND":0.005}


In [10]:
std::vector<double> compress_ratio;
std::vector<int>eb_type;
std::vector<int>alg_type;
for(int i = 0; i<nentries;i++){
    nlohmann::json jobj = nlohmann::json::parse(params(i));
    int alg_option = jobj["ALGO_OPTIONS"];
    int eb_option = jobj["EB_OPTIONS"];
    double comp_ratio = jobj["COMPRESSION_RATIO"];
    compress_ratio.push_back(comp_ratio);
    eb_type.push_back(eb_option);
    alg_type.push_back(alg_option);
}



In [11]:
//Draw the TGraph first...
TGraph *g = new TGraph(compress_ratio.size(),compress_ratio.data());

In [12]:
g->SetTitle("Compression Ratio; DataSets; Compression Ratio");
g->Draw("AC*");

Info in <TCanvas::MakeDefCanvas>:  created default TCanvas with name c1


In [13]:
c1->SaveAs("compression_ratio.png");

Info in <TCanvas::Print>: png file compression_ratio.png has been created


In [14]:
TH1D* h = new TH1D("comp_hist","comp_hist",140,0,14000);
for(size_t i = 0;i<compress_ratio.size();i++){
    h->Fill(compress_ratio[i]);
}

auto c = new TCanvas("c","c",750,750);
c->cd();
h->Draw("hist");
c->SaveAs("compression_histogram.png");
c->SaveAs("compression_histogram.root");

Info in <TCanvas::Print>: png file compression_histogram.png has been created
Info in <TCanvas::SaveAs>: ROOT file compression_histogram.root has been created


In [15]:
TH2D* h2 = new TH2D("comp_dist","comp_dist",5,0,5,3,0,2);
for(size_t i = 0;i<compress_ratio.size();i++){
    h2->Fill(eb_type[i],alg_type[i],compress_ratio[i]);
}
h2->Scale(1.0/compress_ratio.size());
h2->GetXaxis()->SetTitle("Error Bound Type");
h2->GetYaxis()->SetTitle("Algo Options");
h2->GetZaxis()->SetTitle("Normalized Compression ratio");

In [17]:
auto c2 = new TCanvas("c2","c2",950,950);
c2->cd();
h2->Draw("colz");
h2->SetStats(0);
c2->SaveAs("compression_2Dhistogram.png");
c2->SaveAs("compression_2Dhistogram.root");

Warning in <TCanvas::Constructor>: Deleting canvas with same name: c2
Info in <TCanvas::Print>: png file compression_2Dhistogram.png has been created
Info in <TCanvas::SaveAs>: ROOT file compression_2Dhistogram.root has been created
